In [1]:
import os
from libv3.utils import *
import pandas as pd
import json

In [2]:
############ configuration ################
############################################

CODE, BEHAVIOUR, THREAD, VER = get_config()   ### config stored in libv3/exp_config.txt

base_dir = '../trace_data' ### can be replaced with 'csv', 'exe_plot', 'histogram'
log_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR}'

print(log_path)

#### file to display
trace_file = 0

print('file number:', trace_file)

CODE: theft_protection
BEHAVIOUR: faulty_data
THREAD: single
VER: 3
../trace_data/theft_protection/single_thread/version_3/faulty_data
file number: 0


In [3]:
######### get paths #######################
paths_log, paths_traces, varlist_path, paths_label = get_paths(log_path)

### remove.Ds_store from all lists
paths_log = [x for x in paths_log if '.DS_Store' not in x]
paths_traces = [x for x in paths_traces if '.DS_Store' not in x]
varlist_path = [x for x in varlist_path if '.DS_Store' not in x]
paths_label = [x for x in paths_label if '.DS_Store' not in x]

paths_log.sort()
paths_traces.sort()
varlist_path.sort()

print(paths_log)
print(paths_traces)
print(varlist_path)
print(paths_label)

[]
['../trace_data/theft_protection/single_thread/version_3/faulty_data/trace_trial1', '../trace_data/theft_protection/single_thread/version_3/faulty_data/trace_trial2', '../trace_data/theft_protection/single_thread/version_3/faulty_data/trace_trial3']
['../trace_data/theft_protection/single_thread/version_3/faulty_data/varlist_trial1.json', '../trace_data/theft_protection/single_thread/version_3/faulty_data/varlist_trial2.json', '../trace_data/theft_protection/single_thread/version_3/faulty_data/varlist_trial3.json']
['../trace_data/theft_protection/single_thread/version_3/faulty_data/labels/trace_trial1_labels.json', '../trace_data/theft_protection/single_thread/version_3/faulty_data/labels/trace_trial2_labels.json', '../trace_data/theft_protection/single_thread/version_3/faulty_data/labels/trace_trial3_labels.json']


In [4]:
############# check varlist is consistent ############
############# only for version 3 ######################

if VER == 3:
    check_con, _ = is_consistent(varlist_path)

    if check_con != False:
        to_number = read_json(varlist_path[0])
        from_number = mapint2var(to_number)
    else:
        print('varlist is not consistent')


varlist 1 is consistent with varlist 0
varlist 2 is consistent with varlist 0


In [5]:
############ Get variable list ######################
sorted_keys = list(from_number.keys())
sorted_keys.sort()
var_list = [from_number[key] for key in sorted_keys]   ### get the variable list
# print(var_list)

## Trace Plots

### Process Traces

In [6]:
########## process the traces ###########
col_data = preprocess_traces(paths_traces, var_list)   ### in the format (trace_name, x_data, y_data, y_labels, trace_path) 

../trace_data/theft_protection/single_thread/version_3/faulty_data/trace_trial1 trace_trial1
../trace_data/theft_protection/single_thread/version_3/faulty_data/trace_trial2 trace_trial2
../trace_data/theft_protection/single_thread/version_3/faulty_data/trace_trial3 trace_trial3


### Generate plot trace data

In [7]:
############### preprocessing data to plot using plotly ##############
'''
Restructure the data in dictionary with (keys,value) pair :-  (time, timestamps) , (trace_name, trace)
'''
all_df = get_dataframe(col_data)   

In [8]:
all_df[2]

,time,trace_trial3
0,1369,0
1,1397,1
2,1418,2
3,1423,3
4,1931,4
...,...,...
2768,533325,15
2769,533330,6
2770,533357,7
2771,534374,8


In [9]:

### plot single trace
for i, df in enumerate(all_df):
    
    if i == trace_file:
        trace_obj = plot_single_trace(df, var_list, with_time=False, is_xticks=True)
        trace_obj.show()

In [22]:
### get timestamp

timestamp = index2timestamp(all_df[0], 227)
print(timestamp)

48316


## Execution Interval Plots

### Preprocess data

In [9]:
####### preprocessing data ########

#### extract timestamps for each variable and store them in a dictionary along with index values for each variable in event trace

var_timestamps = get_var_timestamps(paths_traces=paths_traces)    #### in format (filename, dict of timestamps and index values)

to_plot = preprocess_variable_plotting(var_timestamps, var_list, from_number, trace_number=trace_file)   ### restructure the data for plotting


length of exe_time: 0
length of exe_time: 0
length of exe_time: 0
length of exe_time: 0
length of exe_time: 0
length of exe_time: 0
length of exe_time: 500
length of exe_time: 500
length of exe_time: 500
length of exe_time: 497
length of exe_time: 105
length of exe_time: 105
length of exe_time: 105
length of exe_time: 81
length of exe_time: 81
length of exe_time: 81


### Generate execution interval plots

In [10]:
### plot the data
plot_list = plot_execution_interval_single(to_plot, is_xticks=False)
for plot in plot_list:
    plot.show()

CODE: theft_protection
BEHAVIOUR: faulty_data
THREAD: single
VER: 3


## Plot Labels

In [12]:
### count and prepare labels to plot
'''
labels are of format [index1, index2, timestamp1, timestamp2, class]
'''
class_count = defaultdict(int)
for i, path in enumerate(paths_label):
    label_content = prepare_gt(path)
    ind, ts, cls = label_content
    # print(ind, ts, cls)
    for c in cls:
        class_count[c]+=1
        
    if i == trace_file:
        print(path)
        toplot_gt = label_content

    print(os.path.split(path)[-1], class_count)

    # break
for key, val in class_count.items():
    print(key, val)

../trace_data/theft_protection/single_thread/version_3/faulty_data/labels/trace1-com_label.json
trace1-com_label.json defaultdict(<class 'int'>, {1: 22, 2: 3})
trace2-sensor_label.json defaultdict(<class 'int'>, {1: 23, 2: 30})
trace3-bitflip_label.json defaultdict(<class 'int'>, {1: 24, 2: 36, 3: 68})
1 24
2 36
3 68


In [13]:
toplot_gt

([(180, 185),
  (199, 204),
  (322, 327),
  (419, 424),
  (516, 521),
  (539, 544),
  (636, 641),
  (733, 738),
  (878, 883),
  (901, 906),
  (920, 924),
  (942, 947),
  (965, 970),
  (1110, 1115),
  (1218, 1225),
  (1602, 1607),
  (1625, 1630),
  (1822, 1827),
  (2067, 2074),
  (2092, 2097),
  (2111, 2116),
  (2186, 2191),
  (2305, 2310),
  (2402, 2407),
  (2425, 2430)],
 [(37775, 38831),
  (41993, 43048),
  (67337, 68393),
  (87403, 88459),
  (107472, 108529),
  (112741, 113797),
  (132810, 133867),
  (152877, 153933),
  (182463, 183518),
  (187732, 188789),
  (191951, 193261),
  (197475, 198530),
  (202743, 203799),
  (232341, 233397),
  (254532, 256897),
  (331993, 333050),
  (337263, 338321),
  (377448, 378504),
  (427165, 428485),
  (432701, 433757),
  (436922, 437977),
  (452779, 453835),
  (477115, 478173),
  (497211, 498267),
  (502485, 503541)],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1])

In [10]:
### plot
for i, df in enumerate(all_df):
    if i == trace_file:
        plot_single_trace(df, var_list, with_time=False, is_xticks=True, ground_truths=toplot_gt)

## Generate train data

In [ ]:
'''
Add the indices for the interval in 'normal_seq_inter' that show correct behvaiour in the traces. 
The format is as follows:
path_traces = [path1, path2, ...]
normal_seq_inter = [ ( intervals for trace1, ...), 
                    ( intervals for trace2 ...),
                      ...] 
'''

normal_seq_inter = (  ( (0,110), (150,900), (1250,2000), (2050,2300) ),  ### v3, normal trace0
            )

for p, n_inter in zip(paths_traces, normal_seq_inter):
    trace = read_traces(p)
    train_data_path = os.path.join(os.path.dirname(p), 'train_data')

    if not os.path.exists(train_data_path):
        os.makedirs(train_data_path)

    for i, inter in enumerate(n_inter):
        start, end = inter
        # trace[start:end].to_csv(os.path.join(train_data_path, f'interval_{start}_{end}.csv'), index=False)
        json.dump(trace[start:end], open(os.path.join(train_data_path, f'interval_{start}_{end}.json'), 'w'))

In [ ]:
normal_seq_inter[0]